# Markov Chains for Weather Prediction
## Authors: Eric Roy, Pau de las Heras, Ricard Renalias

Aplication of Markov Chains models for weather Predition in Manresa.

---
### Historical Meteo Data
Retrieve meteorological data from a station near Manresa.

The data is sourced from GenCat Open Data (https://analisi.transparenciacatalunya.cat).

We use the database of the meteorological station XEMA:
https://analisi.transparenciacatalunya.cat/Medi-Ambient/Dades-meteorol-giques-de-la-XEMA/nzvn-apee/about_data.

The nearest meteorological station to Manresa is "Sant Salvador de Guardiola" (Code: CL).

We analyze the observed data and focus on the variable "Precipitació" (Code: 35).

A filter is created (https://analisi.transparenciacatalunya.cat/Medi-Ambient/Dades-meteorol-giques-de-la-XEMA/nzvn-apee/explore/query/SELECT%0A%20%20%60id%60%2C%0A%20%20%60codi_estacio%60%2C%0A%20%20%60codi_variable%60%2C%0A%20%20%60data_lectura%60%2C%0A%20%20%60data_extrem%60%2C%0A%20%20%60valor_lectura%60%2C%0A%20%20%60codi_estat%60%2C%0A%20%20%60codi_base%60%0AWHERE%0A%20%20caseless_one_of%28%60codi_estacio%60%2C%20%22CL%22%29%0A%20%20AND%20caseless_one_of%28%60codi_variable%60%2C%20%2235%22%29/page/filter), and the data is exported as a CSV file.

As a result, we obtain:
`dades_meteorologiques.csv`


---
We import the data and create a table by assuming that:
- Sunny: If precipitation is 0
- Cloudy: If precipitation is under 5
- Rainy: If precipitation is over 5



In [2]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('dades_meteorologiques.csv')

# Display the first few rows of the dataset
print(df.head())

# Define states based on precipitation levels
def classify_state(precipitation):
    if precipitation == 0:
        return 'Sunny'
    elif precipitation < 5:
        return 'Cloudy'
    else:
        return 'Rainy'

# Apply the classification function to create a new column 'State'
df['State'] = df['VALOR_LECTURA'].apply(classify_state)

# Create the transition matrix
states = ['Sunny', 'Cloudy', 'Rainy']
transition_matrix = pd.DataFrame(0, index=states, columns=states)

# Populate the transition matrix with counts
for i in range(len(df) - 1):
    current_state = df.loc[i, 'State']
    next_state = df.loc[i + 1, 'State']
    transition_matrix.loc[current_state, next_state] += 1

# Convert counts to probabilities
transition_probabilities = transition_matrix.div(transition_matrix.sum(axis=1), axis=0)

# Display the transition probability matrix
print(transition_probabilities)

               ID CODI_ESTACIO  CODI_VARIABLE            DATA_LECTURA  \
0  CL350101090000           CL             35  01/01/2009 12:00:00 AM   
1  CL350101090030           CL             35  01/01/2009 12:30:00 AM   
2  CL350101090100           CL             35  01/01/2009 01:00:00 AM   
3  CL350101090130           CL             35  01/01/2009 01:30:00 AM   
4  CL350101090200           CL             35  01/01/2009 02:00:00 AM   

   DATA_EXTREM  VALOR_LECTURA CODI_ESTAT CODI_BASE  
0          NaN            0.0          V        SH  
1          NaN            0.0          V        SH  
2          NaN            0.0          V        SH  
3          NaN            0.0          V        SH  
4          NaN            0.0          V        SH  
           Sunny    Cloudy     Rainy
Sunny   0.982759  0.016851  0.000390
Cloudy  0.484357  0.505627  0.010015
Rainy   0.333333  0.461240  0.205426


---
Create stationary distribution

In [3]:
# Function to calculate the stationary distribution
def stationary(mat):
    eigvals, eigvecs = np.linalg.eig(mat.T)
    stationary_vector = eigvecs[:, np.isclose(eigvals, 1)]
    stationary_vector = stationary_vector[:, 0]
    stationary_distribution = stationary_vector / stationary_vector.sum()
    return stationary_distribution.real

# Calculate the stationary distribution
stationary_distribution = stationary(transition_probabilities.values)

# Limiting distribution
limiting_distribution = stationary_distribution
limiting_distribution

array([9.65356728e-01, 3.37443513e-02, 8.98920250e-04])

---
Calculate the smallest power m of a transition matrix

In [4]:
# Function to determine the smallest power m of a transition matrix
# with the property that P^m = P^(m+1)
def power_func(matrix):
    for m in range(1, 1001):
        mat1 = np.round(np.linalg.matrix_power(matrix, m), decimals=9)
        mat2 = np.round(np.linalg.matrix_power(matrix, m + 1), decimals=9)
        if np.allclose(mat1, mat2, atol=0):
            return m

# Calculate the power
power = power_func(transition_probabilities.values)
power

22

---
Calculate the limiting Matrix

In [9]:
# The Limiting Matrix is:
P_lim = np.linalg.matrix_power(transition_probabilities.values, 37)
P_lim

array([[9.65356728e-01, 3.37443513e-02, 8.98920250e-04],
       [9.65356728e-01, 3.37443513e-02, 8.98920251e-04],
       [9.65356728e-01, 3.37443513e-02, 8.98920251e-04]])

---
Generate random simulation states

In [10]:
# Set the seed for reproducibility
np.random.seed(123454)

# Simulate results with the three states
simulation_results = np.random.choice(
    a=states,  # Use the states ['Sunny', 'Cloudy', 'Rainy']
    size=100,
    replace=True,
    p=[1/3, 1/3, 1/3]  # Equal probabilities for each state
)

# Absolute frequency distribution
abs_freq_distrib = pd.Series(simulation_results).value_counts()
print("Absolute frequency distribution:")
print(abs_freq_distrib)

# Relative frequency distribution
rel_freq_distrib = pd.Series(simulation_results).value_counts(normalize=True)
print("\nRelative frequency distribution:")
print(rel_freq_distrib)


Absolute frequency distribution:
Sunny     39
Rainy     32
Cloudy    29
Name: count, dtype: int64

Relative frequency distribution:
Sunny     0.39
Rainy     0.32
Cloudy    0.29
Name: proportion, dtype: float64


---
Create a simulation baed on transition probabilities matrix

In [11]:
# Define the initial distribution with equal probabilities for the three states
initial = [1/3, 1/3, 1/3]  # Three states with equal probabilities
print("Initial distribution:", initial)

# Define the states
states = ['Sunny', 'Cloudy', 'Rainy']

# Markov simulation function
def markov(init, matrix, n, labels=None):
    if labels is None:
        labels = list(range(1, len(init) + 1))
    
    # Initialize the simulation list
    simlist = [0] * (n + 1)
    
    # First state based on the initial distribution
    simlist[0] = np.random.choice(len(init), p=init)
    
    # Simulate the Markov chain
    for i in range(1, n + 1):
        simlist[i] = np.random.choice(len(init), p=matrix[simlist[i - 1]])
    
    # Map the states to their labels
    return [labels[state] for state in simlist]

# Transition matrix (already defined as P)
P_matrix = transition_probabilities.values

# Simulate 10,000 Markov chains
replicate_sim = [markov(initial, P_matrix, 100, states) for _ in range(10000)]

# Flatten the results for frequency distribution
flattened_sim = [state for chain in replicate_sim for state in chain]

# Absolute frequency distribution
distr_abs_freq_2 = pd.Series(flattened_sim).value_counts()
print("Absolute frequency distribution:")
print(distr_abs_freq_2)

# Relative frequency distribution
rel_freq_distrib_2 = pd.Series(flattened_sim).value_counts(normalize=True)
print("\nRelative frequency distribution:")
print(rel_freq_distrib_2)


Initial distribution: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
Absolute frequency distribution:
Sunny     960909
Cloudy     43982
Rainy       5109
Name: count, dtype: int64

Relative frequency distribution:
Sunny     0.951395
Cloudy    0.043547
Rainy     0.005058
Name: proportion, dtype: float64
